## Split payments: Smart Signatures with Transaction Groups
#### 06.4.1 Winter School on Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2022-01-22

* Smart Signatures with more than 1 transaction
* Combine conditions across transactions

## Setup
See notebook 04.1, the lines below will always automatically load functions in `algo_util.py`, the 5 accounts and the Purestake credentials

In [ ]:
# Loading shared code and credentials
import sys, os

codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the 3 main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [ ]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk import transaction
from algosdk.transaction import PaymentTxn
from algosdk.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
from algosdk.transaction import LogicSig, LogicSigTransaction

import algosdk.error
import json
import base64
import pandas as pd

In [ ]:
from pyteal import *

In [ ]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])
algod_client.status()["last-round"]

In [ ]:
print(Alice['public'])
print(Bob['public'])
print(Charlie['public'])

## The Split Payment
* Classical Smart Contract
* Example
    * Two business partners agree to split all revenues in a fixed percentage
    * The smart contract is the business acount, into which customers have to pay
    * Both business partners can initiate a payout, but only in the fixed percentage
* Other examples
    * Fixed tax rate
    * Sales commission

#### Step 0: Get the status before the transaction
* Also fund accounts if need be
* https://bank.testnet.algorand.network
* https://testnet.algoexplorer.io/dispenser

In [ ]:
asset_holdings_df2(algod_client, Alice['public'], Bob['public'], ["Alice","Bob"])

#### Step 1a: Write down the conditions as a PyTeal program
* Alice and Bob are business partners
* Alice gets 3/4 of the proceeds
* Bob gets 1/4 of the proceeds

In [ ]:
split_cond = And( 
    Gtxn[0].sender() == Gtxn[1].sender(),                        # Both payments come from the same address
    Gtxn[0].receiver() == Addr(Alice['public']),                 # Payment 0 to Alice
    Gtxn[1].receiver() == Addr(Bob['public']),                   # Payment 1 to Bob
    Gtxn[0].amount() == Int(3) * (Gtxn[0].amount() + Gtxn[1].amount()) / Int(4)    # Alice_amount = 3/4 * Total_amount
    )

fee_cond = And( 
    Gtxn[0].fee() <= Int(1000),                                    # No fee attack
    Gtxn[1].fee() <= Int(1000)                                     # No fee attack
    )

safety_cond = And( 
    Global.group_size() == Int(2),                                  # Exactly 2 transactions
    Gtxn[0].type_enum() == TxnType.Payment,                         # Both are PaymentTxn
    Gtxn[1].type_enum() == TxnType.Payment,
    Gtxn[0].rekey_to() == Global.zero_address(),                    # No rekey attack
    Gtxn[1].rekey_to() == Global.zero_address(),
    Gtxn[0].close_remainder_to() == Global.zero_address(),          # No close_to attack
    Gtxn[1].close_remainder_to() == Global.zero_address()
    )
    
split_pyteal = And(
    split_cond, 
    fee_cond, 
    safety_cond
    )

#### Step 1b: Pyteal -> Teal

In [ ]:
split_teal = compileTeal(split_pyteal, Mode.Signature, version=3)
print(split_teal)

#### Step 1c: Teal -> Bytecode for AVM

In [ ]:
Split = algod_client.compile(split_teal)
Split

### The split payment is now ready
* We only need to communicate the hash to customers

#### Step 2: A customer makes a payment
* Dina buys something from the Alice_Bob_Company
* She pays 5 Algos into the company account

In [ ]:
# Step 2.1: prepare transaction
sp = algod_client.suggested_params()
amt = int(5*1e6)
txn = transaction.PaymentTxn(sender=Dina['public'], sp=sp, 
                             receiver=Split['hash'], amt=amt)

# Step 2.(2+3+4): sign and send and wait ...
stxn = txn.sign(Dina['private'])
txid = algod_client.send_transaction(stxn)
txinfo = wait_for_confirmation(algod_client, txid)

#### Step 3: Payout request
* Alice or Bob (or anybody) can make a payout request
* The only thing that matters is that 3/4 go to Alice and 1/4 goes to Bob
* Consider the TX fees and min holdings in the contract

In [ ]:
### Step 3.1: prepare and create TX group
sp = algod_client.suggested_params()

total_amt = 4.8       # total withdrawl
amt_1 = int(3/4 * 4.8 * 1E6)  # Alice share in microalgos
amt_2 = int(1/4 * 4.8 * 1E6)  # Bob share in microalgos

txn_1 = PaymentTxn(sender=Split['hash'],sp=sp,receiver=Alice['public'],amt=amt_1)
txn_2 = PaymentTxn(sender=Split['hash'],sp=sp,receiver=Bob['public'],amt=amt_2)

gid = transaction.calculate_group_id([txn_1, txn_2])
txn_1.group = gid
txn_2.group = gid

# Step 3.2a ask Smart Signature to sign txn_1
encodedProg = Split['result'].encode()              
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn_1 = LogicSigTransaction(txn_1, lsig)

# Step 3.2b ask Smart Signature to sign txn_2
encodedProg = Split['result'].encode()              
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn_2 = LogicSigTransaction(txn_2, lsig)

# Step 3.3: assemble transaction group and send
signed_group =  [stxn_1, stxn_2]
txid = algod_client.send_transactions(signed_group)

# Step 3.4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

## Appendix: how to calcualte the splitting condition

The splitting condition is written in a strange order

```python
    Gtxn[0].amount() == Int(3) * (Gtxn[0].amount()+Gtxn[1].amount()) / Int(4)  

```

This (mathematically identical) will not work

```python
    Gtxn[0].amount() == Int(3)/Int(4) * (Gtxn[0].amount()+Gtxn[1].amount()) 
```

In [ ]:
a = 750
b = 250

In [ ]:
# Version 1
int(a) == int( int(3 * (int(a) + int(b)) ) / 4)

In [ ]:
# Version 2
int(a) == int(3/4) * ( int(a) + int(b))

In [ ]:
int(3/4) 

#### CONCLUSION: (Py)TEAL uses integer calucaltions for every single step!

## Exercises
1. Change the Smart Signature (and the withdrawl transaction) so that Alice gets 80% and Bob gets 20%
2. Discuss: How would we deal with fractions like 2/3 and 1/3, that cannot be easily expressed in percentages?
3. Discuss: Is there a way to get rid of the whole problem with dividing?